<a href="https://colab.research.google.com/github/arkothiwala/QuoraQuestionPairs/blob/master/Kaggle_tensorflow_speech_recognition_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp ./kaggle.json ~/.kaggle/kaggle.json #Need to upload kaggle.json file from C:\Users\<USERNAME>\.kaggle\

In [2]:
!kaggle competitions download -c tensorflow-speech-recognition-challenge

  0% 0.00/501k [00:00<?, ?B/s]
100% 501k/501k [00:00<00:00, 75.1MB/s]
100% 1.04G/1.04G [00:20<00:00, 36.7MB/s]
100% 1.04G/1.04G [00:20<00:00, 54.6MB/s]
100% 2.46G/2.46G [00:48<00:00, 31.6MB/s]
100% 2.46G/2.46G [00:48<00:00, 54.9MB/s]
  0% 0.00/50.0 [00:00<?, ?B/s]
100% 50.0/50.0 [00:00<00:00, 44.2kB/s]


In [0]:
!7z x train.7z

In [60]:
!7z x test.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2640679130 bytes (2519 MiB)

Extracting archive: test.7z
--
Path = test.7z
Type = 7z
Physical Size = 2640679130
Headers Size = 1345163
Method = Delta LZMA2:24
Solid = +
Blocks = 3

  0%      0% 149 - test/audio/clip_0038827a4.wav                                          0% 318 - test/audio/clip_0083c61d2.wav                                          0% 476 - test/audio/clip_00d109e04.wav                                          0% 640 - test/audio/clip_0119b018c.wav

In [0]:
import os
pip install pydub
from pydub import AudioSegment

In [53]:
root = './train/audio/'
raw_audio = []
category = []
unknown = []
classes = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
unk_classes = []
background = []
from tqdm import tqdm
for cat in tqdm(os.listdir(root)):
  if(cat == '_background_noise_'):
    for fid in os.listdir("{}{}".format(root, cat)):
      if(fid.endswith("wav")):
        audio = AudioSegment.from_wav("{}{}/{}".format(root, cat, fid))
        background.append(audio)
  else:
    for fid in os.listdir("{}{}".format(root, cat)):
      if(fid.endswith("wav")):
        audio = AudioSegment.from_wav("{}{}/{}".format(root, cat, fid))
        if(cat in classes):
          raw_audio.append(audio)
          category.append(cat)
        else:
          unknown.append(audio)
          if(cat not in unk_classes):
            unk_classes.append(cat)

100%|██████████| 31/31 [00:02<00:00, 11.04it/s]


In [50]:
len(raw_audio)

23682

In [59]:
background[5]

In [31]:
len(category)

64727

In [0]:
import pandas as pd
try:
  submission = pd.read_csv('./sample_submission.csv')
except:
  !7z x sample_submission.7z
  submission = pd.read_csv('./sample_submission.csv')

In [39]:
submission.label.value_counts()

silence    158538
Name: label, dtype: int64